## Plain vs finegrained

¿Cuánta ganancia tenemos en la detección?



In [1]:
%load_ext autoreload
%autoreload 2

from hatedetection import load_datasets
import glob
import json

train_dataset, dev_dataset, test_dataset = load_datasets(add_body=True)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


Obs

`text` hace referencia al contexto del tweet del diario
`title` al título de la noticia (extraído del artículo)

In [2]:
!ls ../evaluations/

beto_fine_body.json		  betonews_plain_none.json
beto_fine_none_weighted.json	  betonews_plain_text+body.json
beto_fine_title.json		  betonews_plain_text.json
beto_fine_title_nonweighted.json  beto_plain_none.json
betonews_fine_body.json		  beto_plain_text+body.json
betonews_fine_none.json		  beto_plain_text.json
betonews_fine_text.json		  dev
betonews_fine_title.json	  old


In [42]:
evaluations = {
}
for model_name, path in [
    ('none-plain', "../evaluations/betonews_plain_none.json"),   
    ('tweet-plain', "../evaluations/betonews_plain_text.json"),   
    ('tweet+body-plain', "../evaluations/betonews_plain_text+body.json"),      
    ('none-fine', "../evaluations/betonews_fine_none.json"),   
    ('tweet-fine', "../evaluations/betonews_fine_text.json"),   
    ('tweet+body-fine', "../evaluations/betonews_fine_body.json"),      
    ]:

    with open(path) as f:
        evaluations[model_name] = json.load(f)

for key, evals in evaluations.items():
    print(f"We have {len(evals['metrics'])} {key} evaluations")


We have 10 none-plain evaluations
We have 10 tweet-plain evaluations
We have 9 tweet+body-plain evaluations
We have 10 none-fine evaluations
We have 10 tweet-fine evaluations
We have 15 tweet+body-fine evaluations


In [43]:
import pandas as pd
from hatedetection import extended_hate_categories

df = test_dataset.to_pandas()
df["HATEFUL"] = df[extended_hate_categories].any(1)
df

,id,title,text,article_text,HATEFUL,body,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL
0,397703,Mora Godoy cierra su escuela de tango y remata...,@usuario Ojalá se funda y cague de hambre,Mora Godoy cierra su escuela de tango y remata...,False,Puede marcar con justeza la fecha de inicio de...,0,0,0,0,0,0,0,0,0
1,397704,Mora Godoy cierra su escuela de tango y remata...,@usuario Faltó que se enganchó al hijo de sofo...,Mora Godoy cierra su escuela de tango y remata...,True,Puede marcar con justeza la fecha de inicio de...,0,1,0,0,0,0,0,0,0
2,397705,Mora Godoy cierra su escuela de tango y remata...,"@usuario y bue, que le vaya a llorar a Cristin...",Mora Godoy cierra su escuela de tango y remata...,False,Puede marcar con justeza la fecha de inicio de...,0,0,0,0,0,0,0,0,0
3,397706,Mora Godoy cierra su escuela de tango y remata...,@usuario @usuario Jodete. Macri te hizo conoce...,Mora Godoy cierra su escuela de tango y remata...,False,Puede marcar con justeza la fecha de inicio de...,0,0,0,0,0,0,0,0,0
4,397707,Mora Godoy cierra su escuela de tango y remata...,"@usuario Que se joda, es su gobierno el que el...",Mora Godoy cierra su escuela de tango y remata...,False,Puede marcar con justeza la fecha de inicio de...,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11338,361071,Nahir Galarza sumó a una abogada feminista a s...,@usuario Con esa varita q no rompe un plato ni...,Nahir Galarza sumó a una abogada feminista a s...,False,"La defensa de Nahir Galarza, condenada el 3 de...",0,0,0,0,0,0,0,0,0
11339,361072,Nahir Galarza sumó a una abogada feminista a s...,@usuario No las entiendo a estas pelotudas. So...,Nahir Galarza sumó a una abogada feminista a s...,True,"La defensa de Nahir Galarza, condenada el 3 de...",0,1,0,0,0,0,0,0,0
11340,361073,Nahir Galarza sumó a una abogada feminista a s...,@usuario Lo mató por qué es macho!,Nahir Galarza sumó a una abogada feminista a s...,False,"La defensa de Nahir Galarza, condenada el 3 de...",0,0,0,0,0,0,0,0,0
11341,361074,Nahir Galarza sumó a una abogada feminista a s...,@usuario Cuando sos asesina tenés que hacerte ...,Nahir Galarza sumó a una abogada feminista a s...,False,"La defensa de Nahir Galarza, condenada el 3 de...",0,0,0,0,0,0,0,0,0


In [44]:
df_pred

,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
11338,False,False,False,False,False,False,False,False,False
11339,False,False,False,False,False,False,False,False,False
11340,False,False,True,False,False,False,False,False,False
11341,False,False,False,False,False,False,False,False,False


In [45]:
import numpy as np
from hatedetection import extended_hate_categories
from sklearn.metrics import precision_recall_fscore_support

plain = ['none-plain', 'tweet-plain', 'tweet+body-plain']
fine = ['none-fine', 'tweet-fine', 'tweet+body-fine']

data = []

labels = df["HATEFUL"]
for context in plain:
    print(context)
    for prediction in evaluations[context]["predictions"]:
        preds = np.array(prediction).argmax(1) 
        precision, recall, fscore, support = precision_recall_fscore_support(labels, preds)

        data.append({
            "mode": "plain",
            "context": context.split("-")[0],
            "precision": precision[1],
            "recall": recall[1],
            "F1": fscore[1],
            "Macro F1": fscore.mean()
        })

for context in fine:
    print(context)
    for prediction in evaluations[context]["predictions"]:
        df_pred = pd.DataFrame(prediction, columns=extended_hate_categories) > 0
        preds = df_pred[extended_hate_categories].any(1).values
        precision, recall, fscore, support = precision_recall_fscore_support(labels, preds)
        data.append({
            "mode": "fine",
            "context": context.split("-")[0],
            "precision": precision[1],
            "recall": recall[1],
            "F1": fscore[1],
            "Macro F1": fscore.mean()
        })
        

none-plain
tweet-plain
tweet+body-plain
none-fine
tweet-fine
tweet+body-fine


In [46]:
results_df = pd.DataFrame(data)

results_df

,mode,context,precision,recall,F1,Macro F1
0,plain,none,0.738079,0.594324,0.658446,0.800728
1,plain,none,0.699498,0.620479,0.657623,0.798726
2,plain,none,0.721233,0.585977,0.646607,0.793683
3,plain,none,0.727397,0.590985,0.652134,0.796909
4,plain,none,0.724138,0.595993,0.653846,0.797711
...,...,...,...,...,...,...
59,fine,tweet+body,0.751149,0.636617,0.689157,0.817934
60,fine,tweet+body,0.715461,0.677240,0.695826,0.820187
61,fine,tweet+body,0.736453,0.665554,0.699211,0.822899
62,fine,tweet+body,0.740373,0.663328,0.699736,0.823336


In [60]:
show_df = results_df.groupby(["context", "mode"]).mean().T.loc[:, [('none', 'plain'), ('none', 'fine'), ('tweet', 'plain'), ('tweet', 'fine'),  ('tweet+body', 'plain'), ('tweet+body', 'fine')]]

show_df

context        none               tweet           tweet+body          
mode          plain      fine     plain      fine      plain      fine
precision  0.718280  0.710773  0.748445  0.758727   0.727374  0.739945
recall     0.601669  0.636283  0.653255  0.667168   0.640759  0.659692
F1         0.654535  0.671018  0.697287  0.709826   0.680592  0.697235
Macro F1   0.797827  0.806422  0.822260  0.829532   0.812305  0.821928

In [63]:
pd.options.display.float_format = '{:.3f}'.format

print(show_df.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
context & \multicolumn{2}{l}{none} & \multicolumn{2}{l}{tweet} & \multicolumn{2}{l}{tweet+body} \\
mode & plain &  fine & plain &  fine &      plain &  fine \\
\midrule
precision & 0.718 & 0.711 & 0.748 & 0.759 &      0.727 & 0.740 \\
recall    & 0.602 & 0.636 & 0.653 & 0.667 &      0.641 & 0.660 \\
F1        & 0.655 & 0.671 & 0.697 & 0.710 &      0.681 & 0.697 \\
Macro F1  & 0.798 & 0.806 & 0.822 & 0.830 &      0.812 & 0.822 \\
\bottomrule
\end{tabular}

